# RNN for texts
## Preprocessing

In [5]:
# Standardization and tokenizing
dataset = ['The cat sat on the mat.']

def standardize(text):
    text = text.lower()
    punctuation = ['.', ',', ':', ';', '-', '!', '?']
    for symbol in punctuation:
        text = text.replace(symbol, '')
    return text

def tokenize(text):
    text = text.split()
    return text

vocabulary = {}
for text in dataset:
    text = standardize(text)
    tokens = tokenize(text)
    for token in tokens:
        if token not in vocabulary:
            vocabulary[token] = len(vocabulary)
            
print(vocabulary)



{'the': 0, 'cat': 1, 'sat': 2, 'on': 3, 'mat': 4}


In [6]:
import numpy as np

def one_hot_encode(token):
    vector = np.zeros(shape=(len(vocabulary),))
    token_index = vocabulary[token]
    vector[token_index] = 1

As a class

In [8]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)
    
    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()
    
    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict((v, k) for k, v in self.vocabulary.items())
    
    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]
    
    def decode(self, int_sequence):
        return " ".join(self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)
    
vectorizer = Vectorizer()
dataset = [
"I write, erase, rewrite",
"Erase again, and then",
"A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
encoded_sentence

[2, 3, 5, 7, 1, 5, 6]

In [9]:
decoded_sentence = vectorizer.decode(encoded_sentence)
decoded_sentence

'i write rewrite and [UNK] rewrite again'

Using keras

In [11]:
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(output_mode='int')

custom preprocessing

In [13]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(output_mode="int",
                                       standardize=custom_standardization_fn,
                                       split=custom_split_fn,
)

dataset = [
"I write, erase, rewrite",
"Erase again, and then",
"A poppy blooms.",
]
text_vectorization.adapt(dataset)
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [14]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)


tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)
i write rewrite and [UNK] rewrite again


## IMDB dataset
https://ai.stanford.edu/~amaas/data/sentiment/

In [18]:
import os, pathlib, shutil, random

base_dir = pathlib.Path('aclimdb')
val_dir = base_dir /'val'
train_dir = base_dir / 'train'

for category in ('neg', 'pos'):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category/ fname)

In [41]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [42]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'The first film had little ambition so nothing sticks to the screen. It was a bad version of \'Back to the Future\' with zero charm. Once accepted that Bill & Ted are nitwits, the joke can only get hammered at the audience for so long before it breaks.<br /><br />This is a surprise. This is your only spoiler warning...<br /><br />By today\'s standards, this is more fun. This was shunned upon release, sad considering that more talent is involved than the first time. We get the photographer of \'Face/Off\', the editor of \'Fugitive\', a production designer from Burton\'s early work, and the sound designer of \'Matrix\'.<br /><br />The writers made up for their shallow first outing with something deep. Since this was shunned by the fanbase and public, the director probably decided the style was too extreme. It\'s not, it fits the material. Like \'Death Becomes Her

One word (1gram) for bag-of-words

In [44]:
text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [45]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [46]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [47]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                320016    
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 6s 10ms/step - loss: 0.3939 - accuracy: 0.8392 - val_loss: 0.2961 - val_accuracy: 0.8840
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2650 - accuracy: 0.9014 - val_lo

In [48]:
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 6s 7ms/step - loss: 0.2890 - accuracy: 0.8867TA: 1s - ETA: 0s - loss: 0.2888 - accuracy


[0.28895309567451477, 0.886680006980896]

Bigrams

In [49]:
text_vectorization = TextVectorization(ngrams=2,
                                       max_tokens=20000,
                                       output_mode="multi_hot")

text_vectorization.adapt(text_only_train_ds)

binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [50]:
model = get_model()
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("binary_2gram.keras", save_best_only=True) ]

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                320016    
_________________________________________________________________
dropout_4 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 9s 13ms/step - loss: 0.3755 - accuracy: 0.8456 - val_loss: 0.2780 - val_accuracy: 0.8934
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2413 - accuracy: 0.9166 - val_loss: 0.2894 - val_accuracy: 0.8946
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2088 - accuracy: 0.9337 - val_loss: 0.3090 - val_accuracy: 0.8928
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1969 - accuracy: 0.9401 - val_loss: 0.3174 - val_accuracy: 0.8992
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1742 - accuracy: 0.9459 - val_loss: 0.3336 - val_accuracy: 0.8954
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1833 - accuracy: 0.9475 - val_loss: 0.3478 - val_accuracy: 0.8934
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1695 - accuracy: 0.9503 - val_loss: 0.3573 - val_accuracy: 0.8968
Epoch

In [52]:
model.evaluate(binary_2gram_test_ds)

782/782 [==============================] - 6s 7ms/step - loss: 0.3651 - accuracy: 0.8916


[0.3650892972946167, 0.8915600180625916]

BIGRAMS WITH TF-IDF ENCODING

In [53]:
text_vectorization = TextVectorization(ngrams=2,
                                       max_tokens=20000,
                                       output_mode="tf_idf")

In [54]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)

In [55]:
model = get_model()
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("tfidf_2gram.keras", save_best_only=True) ]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                320016    
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 8s 11ms/step - loss: 0.4847 - accuracy: 0.7831 - val_loss: 0.3178 - val_accuracy: 0.8800
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3294 - accuracy: 0.8602 - val_lo

In [56]:
model.evaluate(tfidf_2gram_test_ds)

782/782 [==============================] - 5s 7ms/step - loss: 0.3775 - accuracy: 0.8720


[0.3775244653224945, 0.8719599843025208]

Word sequence

In [57]:
max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(max_tokens=max_tokens,
                                              output_mode="int",
                                              output_sequence_length=max_length)


text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map( lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [58]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens) # Encode the integers into binary 20,000 dimensional vectors.
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
tf.one_hot (TFOpLambda)      (None, None, 20000)       0         
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                5128448   
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)

Epoch 1/10
 61/625 [=>............................] - ETA: 1:10:46 - loss: 0.6916 - accuracy: 0.5282

KeyboardInterrupt: 

In [ ]:
model.evaluate(int_test_ds)

## Word embedding

In [60]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)
model.evaluate(int_test_ds)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         5120000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                73984     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 223s 352ms/step - loss: 0.4901 - accuracy: 0.7744 - val_loss:

[0.6567695140838623, 0.8385199904441833]

Add masking to ignore padding of zeros for short sentences.

In [62]:
# Example
embedding_layer = layers.Embedding(input_dim=10, output_dim=256, mask_zero=True)
some_input = [
    [4, 3, 2, 1, 0, 0, 0],
    [5, 4, 3, 2, 1, 0, 0],
    [2, 1, 0, 0, 0, 0, 0]]
mask = embedding_layer.compute_mask(some_input)
mask

<tf.Tensor: shape=(3, 7), dtype=bool, numpy=
array([[ True,  True,  True,  True, False, False, False],
       [ True,  True,  True,  True,  True, False, False],
       [ True,  True, False, False, False, False, False]])>

In [63]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, 
                            output_dim=256, 
                            mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 256)         5120000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                73984     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.fit(int_train_ds, 
          validation_data=int_val_ds, 
          epochs=10)
model.evaluate(int_train_ds)

Epoch 1/10
625/625 [==============================] - 263s 414ms/step - loss: 0.3930 - accuracy: 0.8246 - val_loss: 0.2897 - val_accuracy: 0.8790
Epoch 2/10
625/625 [==============================] - 317s 508ms/step - loss: 0.2261 - accuracy: 0.9126 - val_loss: 0.3065 - val_accuracy: 0.8752
Epoch 3/10
625/625 [==============================] - 317s 508ms/step - loss: 0.1600 - accuracy: 0.9402 - val_loss: 0.3062 - val_accuracy: 0.8860
Epoch 4/10
625/625 [==============================] - 316s 505ms/step - loss: 0.1178 - accuracy: 0.9592 - val_loss: 0.3406 - val_accuracy: 0.8744
Epoch 5/10
625/625 [==============================] - 291s 466ms/step - loss: 0.0855 - accuracy: 0.9707 - val_loss: 0.3853 - val_accuracy: 0.8746
Epoch 6/10
625/625 [==============================] - 332s 532ms/step - loss: 0.0630 - accuracy: 0.9785 - val_loss: 0.4020 - val_accuracy: 0.8660
Epoch 7/10
229/625 [=========>....................] - ETA: 2:52 - loss: 0.0489 - accuracy: 0.9829